# SIMD

SIMD stands for **"Single Instruction Multiple Data"** and falls into the category of instruction level parallelism (vector instructions). Consider this simple example where `A`, `B`, and `C` are vectors:

In [3]:
function vector_add(A, B, C)
    for i in eachindex(A, B, C)
        @inbounds A[i] = B[i] + C[i]
    end
end

vector_add (generic function with 1 method)


The idea behind SIMD is to perform the add instruction on multiple elements at the same time (instead of separately performing them one after another). The process of splitting up the simple loop addition into multiple vector additions is often denoted as "loop vectorization". Since each vectorized addition happens at instruction level, i.e. within a CPU core, the feature set of the CPU determines how many elements we can process in one go.

Let's check which "advanced vector extensions" (AVX) the system supports.

In [4]:
using CpuId
cpuinfo()

| Cpu Property       | Value                                                      |
|:------------------ |:---------------------------------------------------------- |
| Brand              | AMD EPYC 7742 64-Core Processor                            |
| Vendor             | :AMD                                                       |
| Architecture       | :Zen                                                       |
| Model              | Family: 0x8f, Model: 0x31, Stepping: 0x00, Type: 0x00      |
| Cores              | 64 physical cores, 128 logical cores (on executing CPU)    |
|                    | Hyperthreading hardware capability detected                |
| Clock Frequencies  | Not supported by CPU                                       |
| Data Cache         | Level 1:3 : (32, 512, 16384) kbytes                        |
|                    | 64 byte cache line size                                    |
| Address Size       | 48 bits virtual, 48 bits physical                          |
| SIMD               | 256 bit = 32 byte max. SIMD vector size                    |
| Time Stamp Counter | TSC is accessible via `rdtsc`                              |
|                    | TSC runs at constant rate (invariant from clock frequency) |
| Perf. Monitoring   | Performance Monitoring Counters (PMC) are not supported    |
|                    | CPU supports AMD's Instruction Based Sampling (IBS)        |
| Hypervisor         | No                                                         |


In [5]:
filter(x -> contains(string(x), "AVX"), cpufeatures())

2-element Vector{Symbol}:
 :AVX
 :AVX2

In [6]:
SIZE = 512 * 100

51200

In [7]:
A = rand(Float64, SIZE)
B = rand(Float64, SIZE)
C = rand(Float64, SIZE);

In [9]:
@code_native debuginfo=:none syntax=:intel vector_add(A,B,C)

	.text
	.file	"vector_add"
	.globl	japi1_vector_add_1877           # -- Begin function japi1_vector_add_1877
	.p2align	4, 0x90
	.type	japi1_vector_add_1877,@function
japi1_vector_add_1877:                  # @japi1_vector_add_1877
	.cfi_startproc
# %bb.0:                                # %top
	push	rbp
	.cfi_def_cfa_offset 16
	.cfi_offset rbp, -16
	mov	rbp, rsp
	.cfi_def_cfa_register rbp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	and	rsp, -32
	sub	rsp, 160
	.cfi_offset rbx, -56
	.cfi_offset r12, -48
	.cfi_offset r13, -40
	.cfi_offset r14, -32
	.cfi_offset r15, -24
	vxorpd	xmm0, xmm0, xmm0
	mov	qword ptr [rsp + 64], 0
	#APP
	mov	rax, qword ptr fs:[0]
	#NO_APP
	lea	rcx, [rsp + 32]
	vmovapd	ymmword ptr [rsp + 32], ymm0
	mov	qword ptr [rsp + 128], rsi
	mov	r12, qword ptr [rax - 8]
	mov	qword ptr [rsp + 32], 12
	mov	rax, qword ptr [r12]
	mov	qword ptr [rsp + 40], rax
	mov	qword ptr [r12], rcx
	mov	rdx, qword ptr [rsi]
	mov	rax, qword ptr [rsi + 8]
	mov	rcx, qword ptr [rsi + 16]
	mov

rcx, r15
	mov	esi, 1392
	mov	edx, 16
	mov	r15, rax
	mov	qword ptr [rax - 8], rbx
	mov	qword ptr [rax], r14
	mov	qword ptr [rsp + 56], rax
	mov	rdi, qword ptr [r12 + 16]
	call	rcx
	mov	rsi, qword ptr [rsp + 24]       # 8-byte Reload
	movabs	rcx, 140359223234912
	mov	qword ptr [rax - 8], rbx
	mov	qword ptr [rax], r13
	mov	qword ptr [rsp + 48], rax
	movabs	rdi, 140359293654032
	mov	edx, 4
	mov	qword ptr [rsp + 96], rcx
	mov	qword ptr [rsp + 104], rsi
	mov	qword ptr [rsp + 112], r15
	mov	qword ptr [rsp + 120], rax
	movabs	rax, offset j1_throw_eachindex_mismatch_indices_1878
	lea	rsi, [rsp + 96]
	call	rax
	ud2
.Lfunc_end0:
	.size	japi1_vector_add_1877, .Lfunc_end0-japi1_vector_add_1877
	.cfi_endproc
                                        # -- End function
	.section	".note.GNU-stack","",@progbits


## It's not always so simple: Reduction

In [10]:
function vector_dot(B, C)
    a = zero(eltype(B))
    for i in eachindex(B,C)
        @inbounds a += B[i] * C[i]
    end
    return a
end

vector_dot (generic function with 1 method)

In [12]:
@code_native debuginfo=:none syntax=:intel vector_dot(B, C)

	.text
	.file	"vector_dot"
	.globl	julia_vector_dot_1903           # -- Begin function julia_vector_dot_1903
	.p2align	4, 0x90
	.type	julia_vector_dot_1903,@function
julia_vector_dot_1903:                  # @julia_vector_dot_1903
	.cfi_startproc
# %bb.0:                                # %top
	push	rbp
	.cfi_def_cfa_offset 16
	.cfi_offset rbp, -16
	mov	rbp, rsp
	.cfi_def_cfa_register rbp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	and	rsp, -32
	sub	rsp, 96
	.cfi_offset rbx, -56
	.cfi_offset r12, -48
	.cfi_offset r13, -40
	.cfi_offset r14, -32
	.cfi_offset r15, -24
	vxorpd	xmm0, xmm0, xmm0
	#APP
	mov	rax, qword ptr fs:[0]
	#NO_APP
	mov	rcx, rsp
	vmovapd	ymmword ptr [rsp], ymm0
	mov	r15, qword ptr [rax - 8]
	mov	qword ptr [rsp], 8
	mov	rax, qword ptr [r15]
	mov	qword ptr [rsp + 8], rax
	mov	qword ptr [r15], rcx
	mov	rbx, qword ptr [rdi + 8]
	mov	r12, qword ptr [rsi + 8]
	cmp	rbx, r12
	jne	.LBB0_6
# %bb.1:                                # %L26
	test	rbx, rbx
	je	.LBB0_2
# %bb.3:   

Note the `vaddsd` instruction and usage of `xmmi` registers (128 bit).

How could this loop be vectorized?

In [13]:
function vector_dot_unrolled4(B, C)
    a1 = zero(eltype(B))
    a2 = zero(eltype(B))
    a3 = zero(eltype(B))
    a4 = zero(eltype(B))
    @inbounds for i in 1:4:length(B)-4
        a1 += B[i] * C[i]
        a2 += B[i+1] * C[i+1]
        a3 += B[i+2] * C[i+2]
        a4 += B[i+3] * C[i+3]
    end
    return a1+a2+a3+a4
end

vector_dot_unrolled4 (generic function with 1 method)

In [15]:
@code_native debuginfo=:none syntax=:intel vector_dot_unrolled4(B, C)

	.text
	.file	"vector_dot_unrolled4"
	.globl	julia_vector_dot_unrolled4_1930 # -- Begin function julia_vector_dot_unrolled4_1930
	.p2align	4, 0x90
	.type	julia_vector_dot_unrolled4_1930,@function
julia_vector_dot_unrolled4_1930:        # @julia_vector_dot_unrolled4_1930
	.cfi_startproc
# %bb.0:                                # %top
	push	r14
	.cfi_def_cfa_offset 16
	push	rbx
	.cfi_def_cfa_offset 24
	sub	rsp, 8
	.cfi_def_cfa_offset 32
	.cfi_offset rbx, -24
	.cfi_offset r14, -16
	mov	rdx, qword ptr [rdi + 8]
	mov	r14, rsi
	mov	rbx, rdi
	movabs	rax, offset j_steprange_last_1932
	mov	edi, 1
	mov	esi, 4
	add	rdx, -4
	call	rax
	test	rax, rax
	jle	.LBB0_1
# %bb.2:                                # %L19.preheader
	mov	rcx, qword ptr [rbx]
	mov	rdx, qword ptr [r14]
	add	rax, 3
	vxorpd	xmm0, xmm0, xmm0
	xor	esi, esi
	.p2align	4, 0x90
.LBB0_3:                                # %L19
                                        # =>This Inner Loop Header: Depth=1
	vmovupd	ymm1, ymmword ptr [rcx + 8*rsi]
	

In [16]:
using BenchmarkTools
@btime vector_dot($B, $C);
@btime vector_dot_unrolled4($B, $C);

  45.315 μs (0 allocations: 0 bytes)


  11.372 μs (0 allocations: 0 bytes)


To "force" automatic SIMD vectorization in Julia, you can use the `@simd` macro.

In [17]:
function vector_dot_simd(B, C)
    a = zero(eltype(B))
    @simd for i in eachindex(B,C)
        @inbounds a += B[i] * C[i]
    end
    return a
end

vector_dot_simd (generic function with 1 method)

By using the `@simd` macro, we are asserting several properties of the loop:

* It is safe to execute iterations in arbitrary or overlapping order, with special consideration for reduction variables.
* Floating-point operations on reduction variables can be reordered, possibly causing different results than without `@simd`.

In [18]:
@btime vector_dot_simd($B, $C);

  9.207 μs (0 allocations: 0 bytes)


This is a huge speedup for just a little extra `@simd`!

In [20]:
@code_native debuginfo=:none syntax=:intel vector_dot_simd(B, C)

	.text
	.file	"vector_dot_simd"
	.globl	julia_vector_dot_simd_2203      # -- Begin function julia_vector_dot_simd_2203
	.p2align	4, 0x90
	.type	julia_vector_dot_simd_2203,@function
julia_vector_dot_simd_2203:             # @julia_vector_dot_simd_2203
	.cfi_startproc
# %bb.0:                                # %top
	push	rbp
	.cfi_def_cfa_offset 16
	.cfi_offset rbp, -16
	mov	rbp, rsp
	.cfi_def_cfa_register rbp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	and	rsp, -32
	sub	rsp, 96
	.cfi_offset rbx, -56
	.cfi_offset r12, -48
	.cfi_offset r13, -40
	.cfi_offset r14, -32
	.cfi_offset r15, -24
	vxorpd	xmm0, xmm0, xmm0
	#APP
	mov	rax, qword ptr fs:[0]
	#NO_APP
	mov	rcx, rsp
	vmovapd	ymmword ptr [rsp], ymm0
	mov	r15, qword ptr [rax - 8]
	mov	qword ptr [rsp], 8
	mov	rax, qword ptr [r15]
	mov	qword ptr [rsp + 8], rax
	mov	qword ptr [r15], rcx
	mov	rbx, qword ptr [rdi + 8]
	mov	r12, qword ptr [rsi + 8]
	cmp	rbx, r12
	jne	.LBB0_10
# %bb.1:                                # %L28
	test	rbx, rbx
	j

Note the `vfmadd231pd` instruction and usage of `ymmi` AVX registers (256 bit).

Data types matter:
* Floating-point addition is **non-associative** and the order of operations is important.
* Integer addition is **associative** and the order of operations has no impact.

Let's check what happens for `Int64` input.

In [21]:
B_int = rand(Int64, SIZE)
C_int = rand(Int64, SIZE)
@btime vector_dot($B_int, $C_int);
@btime vector_dot_simd($B_int, $C_int);

  11.732 μs (0 allocations: 0 bytes)


  11.582 μs (0 allocations: 0 bytes)


As expected, there is no difference between the two variants.

### SIMD is hard...

* Autovectorization is a hard problem (it needs to prove a lot of things about the code!)
* Not every code / loop is readily vectorizable
  * Keep your loops simple, e.g. avoid conditionals, control flow, and function calls if possible!
  * Loop length should be countable up front
  * Contiguous data access
  * (Align data structures to SIMD width boundary)

**Keep it simple!**

### [LoopVectorization.jl](https://github.com/JuliaSIMD/LoopVectorization.jl)

Think of `@turbo` as a more sophisticated version of `@simd`. Hopefully, these features will at some point just be part of Julia's compiler.

In [22]:
using LoopVectorization

function vector_dot_turbo(B, C)
    a = zero(eltype(B))
    @turbo for i in eachindex(B,C)
        @inbounds a += B[i] * C[i]
    end
    return a
end

@btime vector_dot_simd($B, $C);
@btime vector_dot_turbo($B, $C);

  9.207 μs (0 allocations: 0 bytes)


  9.197 μs (0 allocations: 0 bytes)


In [24]:
@code_native debuginfo=:none syntax=:intel vector_dot_turbo(B, C)

	.text
	.file	"vector_dot_turbo"
	.section	.rodata.cst32,"aM",@progbits,32
	.p2align	5                               # -- Begin function julia_vector_dot_turbo_5488
.LCPI0_0:
	.quad	0                               # 0x0
	.quad	1                               # 0x1
	.quad	2                               # 0x2
	.quad	3                               # 0x3
	.text
	.globl	julia_vector_dot_turbo_5488
	.p2align	4, 0x90
	.type	julia_vector_dot_turbo_5488,@function
julia_vector_dot_turbo_5488:            # @julia_vector_dot_turbo_5488
	.cfi_startproc
# %bb.0:                                # %top
	push	rbp
	.cfi_def_cfa_offset 16
	.cfi_offset rbp, -16
	mov	rbp, rsp
	.cfi_def_cfa_register rbp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	and	rsp, -32
	sub	rsp, 96
	.cfi_offset rbx, -56
	.cfi_offset r12, -48
	.cfi_offset r13, -40
	.cfi_offset r14, -32
	.cfi_offset r15, -24
	vxorpd	xmm0, xmm0, xmm0
	#APP
	mov	rax, qword ptr fs:[0]
	#NO_APP
	mov	rcx, rsp
	vmovapd	ymmword ptr [rsp], ymm0
	mov	r1

r13
	pop	r14
	pop	r15
	pop	rbp
	.cfi_def_cfa rsp, 8
	vzeroupper
	ret
.LBB0_14:                               # %L13
	.cfi_def_cfa rbp, 16
	mov	rdi, qword ptr [r15 + 16]
	movabs	r14, offset ijl_gc_pool_alloc
	mov	esi, 1392
	mov	edx, 16
	vzeroupper
	call	r14
	movabs	r13, 140359224160656
	mov	rcx, r14
	mov	esi, 1392
	mov	edx, 16
	mov	r14, rax
	mov	qword ptr [rax - 8], r13
	mov	qword ptr [rax], r12
	mov	qword ptr [rsp + 24], rax
	mov	rdi, qword ptr [r15 + 16]
	call	rcx
	movabs	rcx, 140359223234912
	mov	qword ptr [rax - 8], r13
	mov	qword ptr [rax], rbx
	mov	qword ptr [rsp + 16], rax
	movabs	rdi, 140359293654032
	lea	rsi, [rsp + 48]
	mov	edx, 3
	mov	qword ptr [rsp + 48], rcx
	mov	qword ptr [rsp + 56], r14
	mov	qword ptr [rsp + 64], rax
	movabs	rax, offset j1_throw_eachindex_mismatch_indices_5548
	call	rax
	ud2
.Lfunc_end0:
	.size	julia_vector_dot_turbo_5488, .Lfunc_end0-julia_vector_dot_turbo_5488
	.cfi_endproc
                                        # -- End function
	.section	".note.GNU-s

Doesn't make a difference on Hawk in this case. On systems with AVX512, `@turbo` will lead to usage of `zmmi` AVX512 registers! (512 bit)

## Structure of Array vs Array of Structure

Data layout can matter!

In [25]:
# Array of structure
AoS = [complex(rand(),rand()) for i in 1:SIZE]

51200-element Vector{ComplexF64}:
   0.5075789404837432 + 0.8338526936665136im
   0.7585034190717274 + 0.5363852353532658im
 0.002153716287552121 + 0.48428802059930753im
   0.7552295921428014 + 0.08803697653092912im
    0.816076640824964 + 0.08869623356765366im
   0.7308885924063818 + 0.9013778590199374im
   0.3480106496872565 + 0.6861593788311204im
  0.12656976301264555 + 0.14873246950614805im
   0.6696910462562445 + 0.9302349745865396im
   0.8937498764593206 + 0.2032328394088886im
                      ⋮
   0.5957829447761303 + 0.5998718856357098im
   0.9294962592481967 + 0.1303756292307532im
   0.8363863135130595 + 0.11257479142318227im
   0.1833726190784366 + 0.8775735566102862im
  0.30524486663182604 + 0.9289211196118835im
  0.13883270391605318 + 0.7998808014791987im
  0.06650638334366332 + 0.8185494301294298im
  0.44029109359206664 + 0.8358713770244635im
   0.3611027425099437 + 0.9420516039497241im

In [26]:
@btime sum($AoS);

  13.405 μs (0 allocations: 0 bytes)


### [StructArrays.jl](https://github.com/JuliaArrays/StructArrays.jl)

In [27]:
using StructArrays

In [28]:
SoA = StructArray{Complex}((rand(SIZE), rand(SIZE)))

51200-element StructArray(::Vector{Float64}, ::Vector{Float64}) with eltype Complex:
   0.2238233191149307 + 0.63152988916465im
  0.03383895636440504 + 0.5936974024966287im
   0.1655082330388774 + 0.48002928376595355im
  0.45224841249963776 + 0.33095531623338603im
   0.3086465512713442 + 0.16992034209887974im
  0.44388797924833245 + 0.39471089569155904im
   0.7153846872087188 + 0.7940975383771598im
   0.8275513019371723 + 0.45879827074676316im
   0.5166779635059736 + 0.9833850301607672im
 0.025920582783515944 + 0.9713645187611175im
                      ⋮
   0.3802015638931473 + 0.20699376629192556im
   0.9227203655772644 + 0.29574121877672455im
   0.5220149560765169 + 0.87945137074176im
   0.2899383693589963 + 0.06266127789531206im
   0.6014174530170756 + 0.035213897011814654im
  0.13125759475460708 + 0.3496433679196438im
   0.8035383547430809 + 0.011391432038870053im
  0.14008870938244744 + 0.2717365723684949im
  0.04564653900800397 + 0.1412352321987389im

In [29]:
@btime sum($SoA);

  9.808 μs (0 allocations: 0 bytes)


**Resources:**

* [LoopVectorization.jl video on youtube](https://www.youtube.com/watch?v=qz2kJdVDWi0)
* [SIMD and SIMD-intrinsics in Julia](http://kristofferc.github.io/post/intrinsics/)
* [Optimizing Serial Code](https://mitmath.github.io/18337/lecture2/optimizing)